Name - Rutik Kothwala
UID - U01868702
Subject - CS668, Analytics Capstone

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.util import ngrams
import string
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim.models import Word2Vec
from gensim.models import Word2Vec, KeyedVectors
import nltk
import time
import re
from gensim.utils import simple_preprocess

In [2]:
drugs_reviews= pd.read_csv("C:/Users/rutik/Downloads/drugs_reviews.csv") # loading the preprocessed reviews saved dataset

In [4]:
# Remove rows with NaN values in the 'preprocessed_review' column
drugs_reviews.dropna(subset=['preprocessed_review'], inplace=True)

# Reset the index if needed
drugs_reviews.reset_index(drop=True, inplace=True)

In [5]:
drugs_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211455 entries, 0 to 211454
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           211455 non-null  int64  
 1   drugName             211455 non-null  object 
 2   condition            211455 non-null  object 
 3   review               211455 non-null  object 
 4   rating               211455 non-null  float64
 5   date                 211455 non-null  object 
 6   usefulCount          211455 non-null  int64  
 7   preprocessed_review  211455 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 12.9+ MB


In [6]:
# Convert rating column to positive (1) or negative (0)
drugs_reviews['sentiment'] = drugs_reviews['rating'].apply(lambda x: 1 if x >= 6 else 0)

In [7]:
X = drugs_reviews.drop(['review','rating','date','sentiment'],axis=1)
y = drugs_reviews['sentiment'].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Modelling

# TfIDF

In [9]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 4), max_features=5000)


X_tfidf_train = tfidf_vectorizer.fit_transform(X_train['preprocessed_review'])


X_tfidf_test = tfidf_vectorizer.transform(X_test['preprocessed_review'])

In [ ]:
counter = Counter(y_train)
print('Before', counter)

# oversampling the train dataset using SMOTE
smt = SMOTE()


X_train_sm, y_train_sm = smt.fit_resample(X_tfidf_train, y_train)

counter = Counter(y_train_sm)
print('After',counter)

Before Counter({1: 102434, 0: 43914})
After Counter({1: 102434, 0: 102434})


Random Forest

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_sm, y_train_sm)

RandomForestClassifier(random_state=42)

In [ ]:
rf_accuracy = rf_model.score(X_tfidf_test, y_test)
print('Random Forests Accuracy:', rf_accuracy)

Random Forests Accuracy: 0.911419044495463


In [ ]:
y_pred = rf_model.predict(X_tfidf_test)
classification_rep = classification_report(y_test, y_pred)
print("Classification Report for Random Forest Classifier:")
print(classification_rep)

Classification Report for Random Forest Classifier:
              precision    recall  f1-score   support

           0       0.91      0.79      0.84     11099
           1       0.91      0.97      0.94     25489

    accuracy                           0.91     36588
   macro avg       0.91      0.88      0.89     36588
weighted avg       0.91      0.91      0.91     36588



In [ ]:
# Random Forest is the best model with TF-IDF with accuracy of 0.91

In [ ]:
X_test['model2'] = y_pred

In [ ]:
X_test.to_csv('df_model2.csv')

In [ ]:
X_test.to_csv('C:/Users/rutik/Downloads/df_model2.csv')

In [1]:
# saving the result of the best model  as it will be needed for recommendation score and we don't have to re run the model as it is taking a lot time to run.

XGBoost

In [ ]:
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train_sm, y_train_sm)
xgb_accuracy = xgb_model.score(X_tfidf_test, y_test)
print('XGBoost Accuracy:', xgb_accuracy)

XGBoost Accuracy: 0.8035148136000875


In [ ]:
y_pred2 = xgb_model.predict(X_tfidf_test)
classification_rep = classification_report(y_test, y_pred2)
print("Classification Report for XGBoost:")
print(classification_rep)

Classification Report for XGBoost:
              precision    recall  f1-score   support

           0       0.67      0.71      0.69     11099
           1       0.87      0.85      0.86     25489

    accuracy                           0.80     36588
   macro avg       0.77      0.78      0.77     36588
weighted avg       0.81      0.80      0.81     36588



Naive Bayes

In [10]:
nb_model = MultinomialNB()
nb_model.fit(X_tfidf_train, y_train)
nb_accuracy = nb_model.score(X_tfidf_test, y_test)
print('Naive Bayes Accuracy:', nb_accuracy)

Naive Bayes Accuracy: 0.7968834976708993


In [11]:
y_pred1 = nb_model.predict(X_tfidf_test)
classification_rep = classification_report(y_test, y_pred1)
print("Classification Report for Naive Bayes Classifier:")
print(classification_rep)

Classification Report for Naive Bayes Classifier:
              precision    recall  f1-score   support

           0       0.80      0.44      0.57     12796
           1       0.80      0.95      0.87     29495

    accuracy                           0.80     42291
   macro avg       0.80      0.70      0.72     42291
weighted avg       0.80      0.80      0.78     42291



# Bag-Of_words

In [15]:
# Create the bag-of-words model
bow_model = []

# Create a set to store the vocabulary
vocab = set()

for text in drugs_reviews['review']:
   
    word_counts = {}

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    
    vocab.update(tokens)

    # Count the occurrences of each word
    for word in tokens:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

    # Add the word counts to the bag-of-words model
    bow_model.append(word_counts)

In [16]:
vectorizer = CountVectorizer()

In [17]:
vectorizer.fit(X_train['preprocessed_review'])
bow_model = vectorizer.transform(X_train['preprocessed_review'])
bow_model_test = vectorizer.transform(X_test['preprocessed_review'])

In [18]:
X_train_Bow = bow_model

In [22]:
X_train_sm, y_train_sm = smt.fit_resample(X_train_Bow, y_train)

counter = Counter(y_train_sm)
print('After',counter)

After Counter({1: 102485, 0: 102485})


In [24]:
X_test_bow = bow_model_test

Random Forest

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_sm, y_train_sm)
rf_accuracy = rf_model.score(X_test_bow, y_test)
print('Random Forests Accuracy:', rf_accuracy)

Random Forests Accuracy: 0.8935170001093254


In [ ]:
y_pred_3 = rf_model.predict(X_test_bow)
classification_rep = classification_report(y_test, y_pred_3)
print("Classification Report for Random Forest Classifier:")
print(classification_rep)

Classification Report for Random Forest Classifier:
              precision    recall  f1-score   support

           0       0.87      0.77      0.81     11099
           1       0.90      0.95      0.93     25489

    accuracy                           0.89     36588
   macro avg       0.88      0.86      0.87     36588
weighted avg       0.89      0.89      0.89     36588



In [ ]:
# Random Forest is the best model with Bag - of - Words with accuracy of 0.89

In [ ]:
df_model2= pd.read_csv("C:/Users/rutik/Downloads/df_model2.csv")

In [ ]:
df_model2['model3'] = y_pred_3

In [ ]:
df_model2.to_csv('df_model3.csv')

In [ ]:
df_model2.to_csv('C:/Users/rutik/Downloads/df_model3.csv')

In [ ]:
df_model3= pd.read_csv("C:/Users/rutik/Downloads/df_model3.csv")

In [ ]:
# saving the result of the best model  as it will be needed for recommendation score and we don't have to re run the model as it is taking a lot time to run.

XGBoost

In [29]:
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train_sm, y_train_sm)
xgb_accuracy = xgb_model.score(X_test_bow, y_test)
print('XGBoost Accuracy:', xgb_accuracy)

XGBoost Accuracy: 0.8112769213949929


In [30]:
y_pred = xgb_model.predict(X_test_bow)
classification_rep = classification_report(y_test, y_pred)
print("Classification Report for XGBoost Classifier:")
print(classification_rep)

Classification Report for XGBoost Classifier:
              precision    recall  f1-score   support

           0       0.75      0.57      0.65     11150
           1       0.83      0.92      0.87     25438

    accuracy                           0.81     36588
   macro avg       0.79      0.74      0.76     36588
weighted avg       0.81      0.81      0.80     36588



Naive Bayes

In [34]:
nb_model = MultinomialNB()
nb_model.fit(X_train_sm, y_train_sm)

nb_accuracy = nb_model.score(X_test_bow, y_test)
print('Naive Bayes Accuracy:', nb_accuracy)

Naive Bayes Accuracy: 0.7689679676396632


In [35]:
y_pred_2 = nb_model.predict(X_test_bow)
classification_rep = classification_report(y_test, y_pred_2)
print("Classification Report for Random Forest Classifier:")
print(classification_rep)

Classification Report for Random Forest Classifier:
              precision    recall  f1-score   support

           0       0.60      0.72      0.65     11150
           1       0.87      0.79      0.83     25438

    accuracy                           0.77     36588
   macro avg       0.73      0.76      0.74     36588
weighted avg       0.78      0.77      0.77     36588



# Word2Vec

In [ ]:
# Skip-gram model (sg = 1)
size = 1000
window = 3
min_count = 1
workers = 3
sg = 1

In [ ]:
OUTPUT_FOLDER = "C:/Users/rutik/Downloads/OUTPUT_FOLDER"

In [ ]:
word2vec_model_file = OUTPUT_FOLDER + 'word2vec_' + str(size) + '.model'
start_time = time.time()


stemmed_tokens = X  


w2v_model = Word2Vec(stemmed_tokens, min_count=min_count, vector_size=size, workers=workers, window=window, sg=sg)
print("Time taken to train word2vec model: " + str(time.time() - start_time))


w2v_model.save(word2vec_model_file)

Time taken to train word2vec model: 0.02284550666809082


In [ ]:
w2v_model.save(word2vec_model_file)

In [ ]:
from gensim.models import Word2Vec
w2v_model = Word2Vec.load(word2vec_model_file)

# Function to average word vectors
def average_word_vectors(tokens, model, vector_size):
    word_vectors = []
    for token in tokens:
        if token in model.wv:
            word_vectors.append(model.wv[token])
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(vector_size)  

In [ ]:
X_vectors_W2v_train = np.array([average_word_vectors(tokens, w2v_model, size) for tokens in X_train['preprocessed_review']])

In [ ]:
X_vectors_W2v_test = np.array([average_word_vectors(tokens, w2v_model, size) for tokens in X_test['preprocessed_review']])

In [ ]:
X_train_sm, y_train_sm = smt.fit_resample(X_vectors_W2v_train, y_train)

counter = Counter(y_train_sm)
print('After',counter)

After Counter({1: 102434, 0: 102434})


Random Forest

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_sm, y_train_sm)
rf_accuracy = rf_model.score(X_vectors_W2v_test, y_test)
print('Random Forests Accuracy:', rf_accuracy)

Random Forests Accuracy: 0.8535312124193725


In [ ]:
y_pred5 = rf_model.predict(X_vectors_W2v_test)
classification_rep = classification_report(y_test, y_pred5)
print("Classification Report for Random Forest Classifier:")
print(classification_rep)

Classification Report for Random Forest Classifier:
              precision    recall  f1-score   support

           0       0.78      0.72      0.75     11099
           1       0.88      0.91      0.90     25489

    accuracy                           0.85     36588
   macro avg       0.83      0.82      0.82     36588
weighted avg       0.85      0.85      0.85     36588



In [ ]:
# So Random Forest is the best model with Word2Vec with accuracy of 0.85

In [ ]:
# saving the result of the best model  as it will be needed for recommendation score and we don't have to re run the model as it is taking a lot time to run

In [ ]:
df_model3= pd.read_csv("C:/Users/rutik/Downloads/df_model3.csv")

In [ ]:
df_model3['model4'] = y_pred5

In [ ]:
df_model3.to_csv('df_model4.csv')

In [ ]:
df_model3.to_csv('C:/Users/rutik/Downloads/df_model4.csv')

In [ ]:
df_model4= pd.read_csv("C:/Users/rutik/Downloads/df_model4.csv")

XGBoost

In [ ]:
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train_sm, y_train_sm)
xgb_accuracy = xgb_model.score(X_vectors_W2v_test,y_test)
print('XGBoost Accuracy:', xgb_accuracy)

XGBoost Accuracy: 0.6660653766262162


In [ ]:
y_pred6 = xgb_model.predict(X_vectors_W2v_test)
classification_rep = classification_report(y_test, y_pred6)
print("Classification Report for XGBoost Classifier:")
print(classification_rep)

Classification Report for XGBoost Classifier:
              precision    recall  f1-score   support

           0       0.46      0.61      0.53     11099
           1       0.80      0.69      0.74     25489

    accuracy                           0.67     36588
   macro avg       0.63      0.65      0.63     36588
weighted avg       0.70      0.67      0.68     36588



It can bee seen that for every feature extraction techniques Random Forest model was best, so we have save the results of Random forest model for each techniques as df_model4 and it will be used while developing drug recommendation score.